# Timer

In [1]:
from time import sleep
from datetime import datetime

month, day, hour, minute = 10, 21, 7, 0

while True:
    dt = datetime.now()
    if dt.month >= month:
        if dt.day > day or dt.month > month:
            break
        if dt.day == day and (dt.hour > hour or (dt.hour == hour and dt.minute > minute)):
            break
    sleep(60)

# Parameter optimization

In [1]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
opt_limit = 300
load = True

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {'RSI': {'timeperiod': [12, 14, 16], 'low_bound': [15, 20, 25, 30, 35]},
              'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                        'slowd_period': [3], 'low_bound': [10, 15, 20, 25, 30]}}

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/4h, trade type is buy
Number of combinations is 75


  0%|                                                                                                                                                                                                   | 0/75 [00:00<?, ?it/s]


Load the datasets...
Binance
ByBit
BinanceFutures


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75/75 [12:57<00:00, 10.37s/it]


# Check local statistics

In [3]:
# 15m/4h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('pct_right_forecast_avg', ascending=False).head(20)

,pattern,RSI_timeperiod,RSI_low_bound,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
32,STOCH_RSI,14,20,9,7,3,20,82.54,76.19,87.30,95.24,95.24,93.65,95.24,95.24,98.41,96.83,95.24,95.24,95.24,95.24,93.65,93.65,95.24,92.06,92.06,90.48,88.89,88.89,87.30,85.71,0.47,0.48,0.80,1.26,1.57,1.69,1.63,1.97,2.04,2.03,1.66,1.81,1.92,1.82,1.86,1.85,1.97,1.86,1.91,1.76,1.53,1.58,1.56,1.59,63,91.865417,1.609167,1377.521250,101.377500
6,STOCH_RSI,12,20,9,7,3,15,82.19,76.71,87.67,93.15,94.52,91.78,94.52,95.89,98.63,97.26,95.89,95.89,95.89,94.52,93.15,95.89,95.89,93.15,93.15,89.04,87.67,89.04,87.67,84.93,0.46,0.46,0.65,1.20,1.45,1.51,1.56,1.84,1.88,1.87,1.63,1.82,1.81,1.82,1.99,1.85,1.94,1.86,1.82,1.63,1.53,1.58,1.55,1.53,73,91.837083,1.551667,1594.107083,113.271667
7,STOCH_RSI,12,20,9,7,3,20,83.33,78.57,88.10,94.05,94.05,91.67,94.05,95.24,97.62,96.43,95.24,95.24,95.24,94.05,92.86,95.24,95.24,92.86,92.86,89.29,88.10,89.29,88.10,84.52,0.47,0.48,0.75,1.22,1.49,1.63,1.62,1.92,1.96,1.93,1.68,1.82,1.90,2.14,2.03,2.12,1.98,1.99,1.91,1.78,1.59,1.62,1.59,1.57,84,91.718333,1.632917,1824.340000,137.165000
31,STOCH_RSI,14,20,9,7,3,15,81.82,74.55,87.27,94.55,94.55,92.73,94.55,94.55,98.18,96.36,94.55,94.55,94.55,94.55,92.73,94.55,94.55,92.73,92.73,90.91,89.09,89.09,89.09,87.27,0.46,0.47,0.65,1.25,1.56,1.63,1.57,1.87,2.04,2.01,1.63,1.81,1.81,1.78,1.66,1.81,1.86,1.86,1.90,1.63,1.51,1.51,1.51,1.55,55,91.668750,1.555833,1191.781250,85.570833
59,STOCH_RSI,16,20,9,7,3,30,83.67,75.51,85.71,93.88,95.92,93.88,93.88,93.88,97.96,95.92,93.88,93.88,93.88,93.88,91.84,91.84,93.88,89.80,89.80,89.80,89.80,87.76,87.76,87.76,0.47,0.47,0.65,1.25,1.57,1.76,1.63,1.80,1.88,1.87,1.63,1.78,1.86,1.70,1.53,1.63,1.82,1.78,1.82,1.61,1.49,1.49,1.24,1.30,49,91.073750,1.501250,1032.613750,73.561250
58,STOCH_RSI,16,20,9,7,3,25,83.33,75.00,85.42,93.75,95.83,93.75,93.75,93.75,97.92,95.83,93.75,93.75,93.75,93.75,91.67,91.67,93.75,89.58,89.58,89.58,89.58,87.50,87.50,87.50,0.47,0.46,0.64,1.23,1.64,1.73,1.62,1.82,1.95,1.95,1.64,1.80,1.84,1.66,1.48,1.60,1.79,1.70,1.81,1.62,1.39,1.49,1.28,1.42,48,90.885000,1.501250,1002.480000,72.060000
9,STOCH_RSI,12,20,9,7,3,30,84.27,78.65,87.64,93.26,93.26,91.01,93.26,94.38,96.63,95.51,94.38,94.38,93.26,92.13,91.01,93.26,93.26,91.01,92.13,88.76,87.64,88.76,87.64,84.27,0.50,0.49,0.81,1.24,1.45,1.63,1.63,1.87,1.90,1.88,1.71,1.81,1.92,2.17,2.03,2.12,1.99,2.00,1.91,1.76,1.61,1.61,1.56,1.55,89,90.823333,1.631250,1853.276667,145.181250
8,STOCH_RSI,12,20,9,7,3,25,84.09,78.41,87.50,93.18,93.18,90.91,93.18,94.32,96.59,95.45,94.32,94.32,93.18,92.05,90.91,93.18,93.18,90.91,92.05,88.64,87.50,88.64,87.50,84.09,0.49,0.49,0.81,1.22,1.49,1.63,1.62,1.92,1.96,1.93,1.72,1.82,1.90,2.14,2.03,2.12,1.98,1.99,1.91,1.78,1.59,1.62,1.59,1.57,88,90.720000,1.638333,1823.360000,144.173333
57,STOCH_RSI,16,20,9,7,3,20,82.98,74.47,85.11,93.62,95.74,93.62,93.62,93.62,97.87,95.74,93.62,93.62,93.62,93.62,91.49,91.49,93.62,89.36,89.36,89.36,89.36,87.23,87.23,87.23,0.47,

# Check global statistics

In [2]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            5                  4                  3                  10                            75.759735   
                                                                   5                  10                            83.794091   
               35            11                 3                  9                  20                            66.518698   
               25            7                  3                  5                  10                            88.219697   
12             20            7                  3                  5                  10                            86.237538   
               30            11                 3                  9                  25                            67.022344   
               25            5                  4                  5                  10                            79.443447   
14             30            5                  4                  5                  10                            72.603182   
12             20            7                  4                  5                  10                            59.343523   
16             25            9                  3                  5                  10                            73.011894   
                             7                  3                  7                  10                            56.439394   
12             20            7                  3                  7                  10                            65.530303   
16             25            7                  4                  5                  10                            54.286970   
                             5                  3                  7                  10                            58.333333   
12             20            5                  3                  7                  10                            58.333333   
                             9                  3                  5                  10                            73.201098   
               30            5                  4                  5                  10                            69.466932   
16             25            7                  2                  5                  10                            77.102348   
               35            5                  4                  5                  10                            69.892652   
14             20            5                  5                  5                  10                            75.757576   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            5                  4                  3                  10                         0.843144   
                                                                   5                  10                         1.436705   
               35            11                 3                  9                  20                         0.455156   
               25            7                  3                  5                  10                         1.885038   
12             20            7                  3                  5                  10                         1.803295   
               30            11                 3                  9                  25                         0.418698   
               25            5                  4                  5                  10                         1.366742   
14             30            5                  4                  5                  10                         0.644697   
12            

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '4h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'RSI': {'timeperiod': [14], 'low_bound': [20]},
                  'STOCH': {'fastk_period': [9], 'slowk_period': [7],
                            'slowd_period': [3], 'low_bound': [20]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)